# Table of Contents
 <p><div class="lev1"><a href="#Summary"><span class="toc-item-num">1&nbsp;&nbsp;</span>Summary</a></div><div class="lev1"><a href="#Imports"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></div><div class="lev1"><a href="#Load-data"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load data</a></div><div class="lev1"><a href="#Add-SIFTS"><span class="toc-item-num">4&nbsp;&nbsp;</span>Add SIFTS</a></div><div class="lev2"><a href="#Mutation"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Mutation</a></div><div class="lev2"><a href="#Pfam-clan"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Pfam clan</a></div><div class="lev2"><a href="#Partner-chain"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Partner chain</a></div><div class="lev2"><a href="#Summary"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Summary</a></div><div class="lev1"><a href="#Save-to-database"><span class="toc-item-num">5&nbsp;&nbsp;</span>Save to database</a></div>

# Summary

Process the SKEMPI dataset.

----

# Imports

In [1]:
%run imports.ipynb

2016-07-17 21:06:36.205748


In [2]:
NOTEBOOK_NAME = 'skempi'
os.makedirs(NOTEBOOK_NAME, exist_ok=True)
os.environ['NOTEBOOK_NAME'] = NOTEBOOK_NAME

os.environ['DB_PORT'] = '8308'

In [3]:
%run mysqld.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-07-17 21:06:36.419724


In [4]:
# Start a database connection
db_remote = datapkg.MySQL(
    os.environ['DATAPKG_CONNECTION_STR'], 
    shared_folder=os.environ['NOTEBOOK_NAME'],
    storage_host=None,
    echo=False,
    db_engine='InnoDB'
)

# Load data

In [5]:
R = 1.9872041 * 10**(-3)  # kcal / K / mol

In [6]:
SKEMPI_DF = pd.read_csv('../downloads/skempi/SKEMPI_1.1.csv', sep=';')

In [7]:
display(SKEMPI_DF.head())
print(SKEMPI_DF.shape[0])

,Protein,Mutation(s)_PDB,Mutation(s)_cleaned,Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_wt (M),Reference,Protein 1,Protein 2,Temperature,kon_mut (M^(-1)s^(-1)),kon_wt (M^(-1)s^(-1)),koff_mut (s^(-1)),koff_wt (s^(-1)),dH_mut (kcal mol^(-1)),dH_wt (kcal mol^(-1)),dS_mut (cal mol^(-1) K^(-1)),dS_wt (cal mol^(-1) K^(-1)),Notes
0,1CSE_E_I,LI45G,LI38G,COR,PI,PI,5.260000e-11,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1CSE_E_I,LI45S,LI38S,COR,PI,PI,8.330000e-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1CSE_E_I,LI45P,LI38P,COR,PI,PI,1.020000e-07,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1CSE_E_I,LI45I,LI38I,COR,PI,PI,1.720000e-10,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1CSE_E_I,LI45D,LI38D,COR,PI,PI,1.920000e-09,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


3047


In [8]:
skempi_df = SKEMPI_DF.copy()

In [9]:
# Number of PDBs with multiple mutations
print(skempi_df[skempi_df['Mutation(s)_PDB'].str.contains(',')].shape[0])

730


In [10]:
def get_pdb_mutation(muts):
    converted_muts = []
    for mut in muts.split(','):
        converted_muts.append(mut[0] + mut[2:])
    return ','.join(converted_muts)


skempi_df['pdb_id'] = skempi_df['Protein'].str[:4].str.lower()
skempi_df['pdb_chain'] = skempi_df['Mutation(s)_PDB'].str[1]
skempi_df['pdb_mutation'] = skempi_df['Mutation(s)_PDB'].apply(get_pdb_mutation)

skempi_df['partner_pdb_chain'] = skempi_df[['Protein', 'pdb_chain']].apply(
    lambda x: local.get_partner_chain(*x), axis=1)

In [11]:
skempi_df['temperature_for_ddg'] = skempi_df['Temperature'].apply(lambda x: float(x.split('(')[0]))
skempi_df['ddg_exp'] = [
    R * T * np.log(Kd_mut / Kd_wt) for (T, Kd_mut, Kd_wt)
    in skempi_df[['temperature_for_ddg', 'Affinity_mut (M)', 'Affinity_wt (M)']].values
]

In [12]:
display(skempi_df.head())
display(skempi_df.dtypes)
print(skempi_df.shape[0])

,Protein,Mutation(s)_PDB,Mutation(s)_cleaned,Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_wt (M),Reference,Protein 1,Protein 2,Temperature,kon_mut (M^(-1)s^(-1)),kon_wt (M^(-1)s^(-1)),koff_mut (s^(-1)),koff_wt (s^(-1)),dH_mut (kcal mol^(-1)),dH_wt (kcal mol^(-1)),dS_mut (cal mol^(-1) K^(-1)),dS_wt (cal mol^(-1) K^(-1)),Notes,pdb_id,pdb_chain,pdb_mutation,partner_pdb_chain,temperature_for_ddg,ddg_exp
0,1CSE_E_I,LI45G,LI38G,COR,PI,PI,5.260000e-11,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1cse,I,L45G,E,294.0,2.248958
1,1CSE_E_I,LI45S,LI38S,COR,PI,PI,8.330000e-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1cse,I,L45S,E,294.0,1.172294
2,1CSE_E_I,LI45P,LI38P,COR,PI,PI,1.020000e-07,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1cse,I,L45P,E,294.0,6.671647
3,1CSE_E_I,LI45I,LI38I,COR,PI,PI,1.720000e-10,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1cse,I,L45I,E,294.0,2.941151
4,1CSE_E_I,LI45D,LI38D,COR,PI,PI,1.920000e-09,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1cse,I,L45D,E,294.0,4.350675


Protein                          object
Mutation(s)_PDB                  object
Mutation(s)_cleaned              object
Location(s)                      object
Hold_out_type                    object
Hold_out_proteins                object
Affinity_mut (M)                float64
Affinity_wt (M)                 float64
Reference                        object
Protein 1                        object
Protein 2                        object
Temperature                      object
kon_mut (M^(-1)s^(-1))          float64
kon_wt (M^(-1)s^(-1))           float64
koff_mut (s^(-1))               float64
koff_wt (s^(-1))                float64
dH_mut (kcal mol^(-1))          float64
dH_wt (kcal mol^(-1))           float64
dS_mut (cal mol^(-1) K^(-1))    float64
dS_wt (cal mol^(-1) K^(-1))     float64
Notes                            object
pdb_id                           object
pdb_chain                        object
pdb_mutation                     object
partner_pdb_chain                object


3047


In [13]:
skempi_df_1_bak = skempi_df.copy()

# Add SIFTS

In [14]:
skempi_df = skempi_df_1_bak.copy()

## Mutation

In [15]:
pdb_ids = sorted(set(skempi_df['pdb_id']))
print(len(pdb_ids))

158


In [16]:
sifts_dfs = {
    pdb_id: ascommon.pdb_tools.sifts.get_sifts_data(pdb_id)
    for pdb_id in pdb_ids
}

In [17]:
sifts_dfs['1a22'].head()

,comments,is_observed,pdb_aa,pdb_chain,pdb_id,pfam_id,resnum,uniprot_aa,uniprot_id,uniprot_position,residx
0,"T,loop",True,F,A,1a22,PF00103,1,F,P01241,27,1
1,"T,loop",True,P,A,1a22,PF00103,2,P,P01241,28,2
2,"T,loop",True,T,A,1a22,PF00103,3,T,P01241,29,3
3,"T,loop",True,I,A,1a22,PF00103,4,I,P01241,30,4
4,"T,loop",True,P,A,1a22,PF00103,5,P,P01241,31,5


In [18]:
assert not any(df is None for df in sifts_dfs.values())

In [19]:
skempi_df[['Protein', 'pdb_chain', 'pdb_mutation']]

,Protein,pdb_chain,pdb_mutation
0,1CSE_E_I,I,L45G
1,1CSE_E_I,I,L45S
2,1CSE_E_I,I,L45P
3,1CSE_E_I,I,L45I
4,1CSE_E_I,I,L45D
5,1CSE_E_I,I,L45E
6,1ACB_E_I,I,L45G
7,1ACB_E_I,I,L45S
8,1ACB_E_I,I,L45P
9,1ACB_E_I,I,L45I


In [20]:
reload(kmtools)
reload(kmtools.pdb_tools.sifts)

<module 'kmtools.pdb_tools.sifts' from '/home/kimlab1/strokach/working/kmtools/kmtools/pdb_tools/sifts.py'>

In [21]:
# Add info to core
def get_sifts_data(pdb_id, pdb_chain, pdb_mutation):
    sifts_df = sifts_dfs[pdb_id]
    try:
        result = kmtools.pdb_tools.sifts.convert_pdb_mutations_to_uniprot(
            pdb_id, pdb_chain, pdb_mutation, sifts_df)
    except kmtools.pdb_tools.sifts.SIFTSError as e:
        logger.error(e)
        result = {}
    return result

results = [get_sifts_data(*x) for x in skempi_df[['pdb_id', 'pdb_chain', 'pdb_mutation']].values]

ERROR:root:PDB AA and UniProt AA are not the same! (1sbn, I, R, 45, None, 0) (R != L (P01051))
ERROR:root:PDB AA and UniProt AA are not the same! (1sib, I, K, 53, None, 0) (K != R (P01051))
ERROR:root:PDB AA and UniProt AA are not the same! (1y1k, I, A, 58, None, 0) (A != T (Q40059))
ERROR:root:PDB AA and UniProt AA are not the same! (1y33, I, P, 58, None, 0) (P != T (Q40059))
ERROR:root:PDB AA and UniProt AA are not the same! (1y34, I, A, 60, None, 0) (A != E (Q40059))
ERROR:root:PDB AA and UniProt AA are not the same! (1y3b, I, S, 60, None, 0) (S != E (Q40059))
ERROR:root:PDB AA and UniProt AA are not the same! (1y4a, I, S, 60, None, 0) (S != E (Q40059))
ERROR:root:PDB AA and UniProt AA are not the same! (1y3c, I, A, 62, None, 0) (A != R (Q40059))
ERROR:root:PDB AA and UniProt AA are not the same! (1y48, I, A, 65, None, 0) (A != R (Q40059))
ERROR:root:PDB AA and UniProt AA are not the same! (1y3d, I, A, 67, None, 0) (A != R (Q40059))
ERROR:root:PDB AA and UniProt AA are not the same!

In [22]:
df = pd.DataFrame(results)
df.head()

,pfam_id_sifts,uniprot_id_sifts,uniprot_mutations_sifts
0,NaN,P01051,L45G
1,NaN,P01051,L45S
2,NaN,P01051,L45P
3,NaN,P01051,L45I
4,NaN,P01051,L45D


In [23]:
skempi_df['uniprot_id'] = df['uniprot_id_sifts']
skempi_df['uniprot_mutation'] = df['uniprot_mutations_sifts']
skempi_df['pfam_id'] = df['pfam_id_sifts']

In [24]:
print2("Number of rows:", skempi_df.shape[0])
print2("Number of missing uniprots:", skempi_df['uniprot_id'].isnull().sum())
print2("Number of missing mutations:", skempi_df['uniprot_mutation'].isnull().sum())
print2("Number of missing pfams:", skempi_df['pfam_id'].isnull().sum())

Number of rows:                                             3047
Number of missing uniprots:                                 459
Number of missing mutations:                                459
Number of missing pfams:                                    956


<em>
NOTE: There are only ~350 missing mutations if you include cases where the UniProt AA and the PDB AA do not match!
</em>

In [25]:
skempi_df[
    (skempi_df['uniprot_id'] == 'P36894') &
    (skempi_df['uniprot_mutation'] == 'A97A,M101M,K102K,Y103Y')
]

,Protein,Mutation(s)_PDB,Mutation(s)_cleaned,Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_wt (M),Reference,Protein 1,Protein 2,Temperature,kon_mut (M^(-1)s^(-1)),kon_wt (M^(-1)s^(-1)),koff_mut (s^(-1)),koff_wt (s^(-1)),dH_mut (kcal mol^(-1)),dH_wt (kcal mol^(-1)),dS_mut (cal mol^(-1) K^(-1)),dS_wt (cal mol^(-1) K^(-1)),Notes,pdb_id,pdb_chain,pdb_mutation,partner_pdb_chain,temperature_for_ddg,ddg_exp,uniprot_id,uniprot_mutation,pfam_id


In [26]:
get_sifts_data('2qja', 'C', 'T74A,L78M,G79K,L80Y')

ERROR:root:PDB AA and UniProt AA are not the same! (2qja, C, T, 74, None, 0) (T != A (P36894))


{}

## Pfam clan

In [28]:
pfam_a_clans = (
    pd.read_sql_table('pfam_a_clans', db_remote.engine, schema='pfam')
)

In [29]:
pfam_a_clans.head()

,pfam_id,clan_id,clan_name,pfam_name,pfam_description
0,PF00389,CL0325,Form_Glyc_dh,2-Hacid_dh,"D-isomer specific 2-hydroxyacid dehydrogenase,..."
1,PF00198,CL0149,CoA-acyltrans,2-oxoacid_dh,2-oxoacid dehydrogenases acyltransferase (cata...
2,PF04029,None,None,2-ph_phosp,2-phosphosulpholactate phosphatase
3,PF03171,CL0029,Cupin,2OG-FeII_Oxy,2OG-Fe(II) oxygenase superfamily
4,PF01073,CL0063,NADP_Rossmann,3Beta_HSD,3-beta hydroxysteroid dehydrogenase/isomerase ...


In [30]:
skempi_df['pfam_clan'] = skempi_df['pfam_id'].map(pfam_a_clans.set_index('pfam_id')['clan_id'])

In [31]:
assert skempi_df['pfam_clan'].notnull().sum() > 100

## Partner chain 

In [32]:
def get_partner_uniprot(partner_chian, sifts_df):
    sifts_df = sifts_df[sifts_df['pdb_chain'] == partner_chian]
    partner_uniprot_ids = sifts_df['uniprot_id'].dropna().drop_duplicates().tolist()
    if len(partner_uniprot_ids) == 0:
        return np.nan
    elif len(partner_uniprot_ids) == 1:
        return partner_uniprot_ids[0]
    else:
        raise Exception(partner_uniprot_ids)

In [33]:
get_partner_uniprot('E', sifts_dfs['1cse'])

'B0FXJ2'

In [34]:
skempi_df['partner_uniprot_id'] = [
    get_partner_uniprot(partner_chain, sifts_dfs[pdb_id])
    for pdb_id, partner_chain
    in skempi_df[['pdb_id', 'partner_pdb_chain']].values
]

## Summary

In [35]:
print2("Number of rows:", skempi_df.shape[0])
print('-' * 80)

print2("Number of missing uniprots:", skempi_df['uniprot_id'].isnull().sum())
print2("Number of missing mutations:", skempi_df['uniprot_mutation'].isnull().sum())
print2("Number of missing uniprots mutations:", 
       skempi_df[['uniprot_id', 'uniprot_mutation']].isnull().any(axis=1).sum())
print('-' * 80)

print2("Number of missing partner uniprots:", skempi_df['partner_uniprot_id'].isnull().sum())
print2("Number of missing partner uniprot mutations:", 
       skempi_df[['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation']].isnull().any(axis=1).sum())
print('-' * 80)

print2("Number of missing pfams:", skempi_df['pfam_id'].isnull().sum())

Number of rows:                                             3047
--------------------------------------------------------------------------------
Number of missing uniprots:                                 459
Number of missing mutations:                                459
Number of missing uniprots mutations:                       459
--------------------------------------------------------------------------------
Number of missing partner uniprots:                         171
Number of missing partner uniprot mutations:                615
--------------------------------------------------------------------------------
Number of missing pfams:                                    956


In [36]:
skempi_df_2_bak = skempi_df.copy()

# Save to database

In [38]:
skempi_df = skempi_df_2_bak.copy()

In [39]:
groupby_columns = [
    'uniprot_id', 'partner_uniprot_id', 'uniprot_mutation'
]
extra_columns = [
    'pdb_id', 'pdb_chain', 'partner_pdb_chain', 'pdb_mutation', 
    'pfam_id', 'pfam_clan',
]
data_columns = [
    'ddg_exp'
]

In [40]:
# Average over duplicate mutations
df = (
    skempi_df
    .groupby(groupby_columns)
    .agg({**{c: lambda x: x.iloc[0] for c in extra_columns}, **{c: 'mean' for c in data_columns}})
    .reset_index()
)

In [41]:
print(skempi_df.shape[0])
print(df.shape[0])

3047
2213


In [42]:
t = db.import_df(
    df[groupby_columns + extra_columns + data_columns],
    'skempi',
)

In [43]:
t.name

'skempi'

In [44]:
db.create_indexes(
    t.name,
    [(['pdb_id', 'pdb_chain', 'partner_pdb_chain', 'pdb_mutation'], True),
     (['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation'], True)]
)

In [45]:
db.add_idx_column(t.name, 'id')

2213